# 🎬 Auto-Dubbing Pipeline v2

영어 영상 → 한국어 더빙 + 자막

**특징:**
- Whisper timestamps로 음성 구간 자동 감지
- 배경음악/효과음 유지 (넷플릭스 스타일)
- 클라우드에서 안정적 처리

**사용법:**
1. 런타임 → 런타임 유형 변경 → **GPU (T4)** 선택
2. 셀 순서대로 실행
3. API 키 입력
4. 유튜브 URL 입력

## 1️⃣ 설치

In [ ]:
# 필수 패키지 설치
!pip install -q openai requests yt-dlp
!apt-get install -y ffmpeg > /dev/null 2>&1

print("✅ 설치 완료!")

## 2️⃣ API 키 설정 ⚠️ 필수!

In [ ]:
#@title API 키 입력
import os
from getpass import getpass

print("API 키를 입력하세요 (입력 시 화면에 표시되지 않음):")
print()

openai_key = getpass("OpenAI API Key: ")
elevenlabs_key = getpass("ElevenLabs API Key: ")

os.environ['OPENAI_API_KEY'] = openai_key
os.environ['ELEVENLABS_API_KEY'] = elevenlabs_key

print()
print("✅ API 키 설정 완료!")

## 3️⃣ 더빙 파이프라인 코드

In [ ]:
import os
import json
import subprocess
import tempfile
from pathlib import Path
from openai import OpenAI
import requests

# 설정
ELEVENLABS_API_KEY = os.environ.get('ELEVENLABS_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# ElevenLabs 목소리
VOICES = {
    "male_1": "pNInz6obpgDQGcFmaJgB",    # Adam
    "male_2": "VR6AewLTigWG4xSOukaG",    # Arnold
    "female_1": "pFZP5JQG7iQjIQuC4Bku",  # Lily
    "female_2": "21m00Tcm4TlvDq8ikWAM",  # Rachel
}

def download_youtube(url, output_dir):
    """유튜브 다운로드"""
    print(f"  URL: {url}")
    cmd = [
        "yt-dlp", "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]",
        "--merge-output-format", "mp4",
        "-o", f"{output_dir}/%(title)s.%(ext)s",
        url
    ]
    subprocess.run(cmd, check=True, capture_output=True)
    files = list(Path(output_dir).glob("*.mp4"))
    if files:
        print(f"  완료: {files[0].name}")
        return str(files[0])
    raise Exception("다운로드 실패")

def extract_audio(video_path, audio_path):
    """오디오 추출 (MP3)"""
    cmd = ["ffmpeg", "-y", "-i", video_path, "-vn", "-acodec", "libmp3lame", "-q:a", "2", audio_path]
    subprocess.run(cmd, capture_output=True, check=True)
    return audio_path

def transcribe_with_timestamps(audio_path):
    """Whisper로 텍스트 + 타임스탬프 추출 (VAD 대체)"""
    print("  Whisper API 호출 중...")
    client = OpenAI()
    
    with open(audio_path, "rb") as f:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=f,
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )
    
    segments = []
    for seg in response.segments:
        if seg.text.strip():  # 텍스트가 있는 세그먼트만
            segments.append({
                'start': round(seg.start, 2),
                'end': round(seg.end, 2),
                'duration': round(seg.end - seg.start, 2),
                'text': seg.text.strip()
            })
    
    print(f"  {len(segments)}개 음성 세그먼트 감지")
    return segments

def translate_segments(segments, tone="formal"):
    """GPT로 번역"""
    print(f"  말투: {tone}")
    client = OpenAI()
    
    tone_desc = {
        "formal": "존댓말 (~습니다/~요)",
        "casual": "반말 (~해/~야)",
        "narration": "나레이션체 (~다/~했다)"
    }.get(tone, "존댓말")
    
    # 배치 처리 (50개씩)
    batch_size = 50
    for i in range(0, len(segments), batch_size):
        batch = segments[i:i+batch_size]
        texts = [{"id": j, "text": s["text"], "duration": s["duration"]} for j, s in enumerate(batch)]
        
        prompt = f"""Translate to Korean ({tone_desc}). Keep translations speakable within the duration.
Return JSON array with "id" and "translation".

{json.dumps(texts, ensure_ascii=False)}"""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        
        content = response.choices[0].message.content
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0]
        elif "```" in content:
            content = content.split("```")[1].split("```")[0]
        
        translations = {item["id"]: item["translation"] for item in json.loads(content.strip())}
        for j, seg in enumerate(batch):
            seg["translated"] = translations.get(j, seg["text"])
        
        print(f"  {min(i+batch_size, len(segments))}/{len(segments)} 번역 완료", end="\r")
    
    print(f"  {len(segments)}개 번역 완료          ")
    return segments

def generate_tts(text, output_path, voice_id):
    """ElevenLabs TTS"""
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    response = requests.post(
        url,
        headers={"xi-api-key": ELEVENLABS_API_KEY, "Content-Type": "application/json"},
        json={"text": text, "model_id": "eleven_multilingual_v2"}
    )
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    return output_path

def process_tts(segments, temp_dir, voice="female_1"):
    """TTS 생성"""
    print(f"  목소리: {voice}")
    voice_id = VOICES.get(voice, VOICES["female_1"])
    
    for i, seg in enumerate(segments):
        tts_path = f"{temp_dir}/tts_{i:04d}.mp3"
        generate_tts(seg["translated"], tts_path, voice_id)
        seg["tts_path"] = tts_path
        print(f"  {i+1}/{len(segments)} TTS 생성", end="\r")
    
    print(f"  {len(segments)}개 TTS 완료          ")
    return segments

def get_audio_duration(path):
    """오디오 길이 측정"""
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration",
         "-of", "default=noprint_wrappers=1:nokey=1", path],
        capture_output=True, text=True
    )
    return float(result.stdout.strip())

def time_stretch_audio(input_path, output_path, ratio):
    """오디오 속도 조절"""
    ratio = max(0.5, min(2.0, ratio))
    cmd = ["ffmpeg", "-y", "-i", input_path, "-filter:a", f"atempo={ratio}", "-vn", output_path]
    subprocess.run(cmd, capture_output=True, check=True)
    return output_path

def sync_tts_to_segments(segments, temp_dir):
    """TTS를 원본 타이밍에 맞게 조절"""
    print("  타이밍 조절 중...")
    for i, seg in enumerate(segments):
        tts_duration = get_audio_duration(seg["tts_path"])
        target_duration = seg["duration"]
        
        if tts_duration > 0 and target_duration > 0:
            ratio = target_duration / tts_duration
            # 25% 이내만 조절
            if 0.75 <= ratio <= 1.25 and abs(ratio - 1.0) > 0.05:
                stretched_path = f"{temp_dir}/stretched_{i:04d}.mp3"
                time_stretch_audio(seg["tts_path"], stretched_path, ratio)
                seg["tts_path"] = stretched_path
    
    print("  타이밍 조절 완료")
    return segments

def mix_audio(original_audio, segments, output_path, original_volume=0.15):
    """원본 오디오 + 더빙 믹싱"""
    print(f"  원본 볼륨: {int(original_volume*100)}%")
    
    if not segments:
        subprocess.run(["ffmpeg", "-y", "-i", original_audio, output_path], capture_output=True)
        return output_path
    
    inputs = ["-i", original_audio]
    filter_parts = [f"[0:a]volume={original_volume}[orig]"]
    overlay_inputs = ["[orig]"]
    
    for i, seg in enumerate(segments):
        inputs.extend(["-i", seg["tts_path"]])
        delay_ms = int(seg["start"] * 1000)
        filter_parts.append(f"[{i+1}:a]adelay={delay_ms}|{delay_ms}[dub{i}]")
        overlay_inputs.append(f"[dub{i}]")
    
    filter_parts.append(f"{''.join(overlay_inputs)}amix=inputs={len(segments)+1}:duration=longest:normalize=0[out]")
    
    cmd = ["ffmpeg", "-y"] + inputs + [
        "-filter_complex", ";".join(filter_parts),
        "-map", "[out]", "-c:a", "libmp3lame", "-q:a", "2", output_path
    ]
    subprocess.run(cmd, capture_output=True, check=True)
    print("  믹싱 완료")
    return output_path

def replace_audio(video_path, audio_path, output_path):
    """영상 오디오 교체"""
    cmd = ["ffmpeg", "-y", "-i", video_path, "-i", audio_path,
           "-c:v", "copy", "-map", "0:v:0", "-map", "1:a:0", "-shortest", output_path]
    subprocess.run(cmd, capture_output=True, check=True)
    return output_path

def generate_srt(segments, output_path):
    """SRT 자막 생성"""
    def fmt_time(s):
        h, m, sec = int(s//3600), int((s%3600)//60), int(s%60)
        ms = int((s%1)*1000)
        return f"{h:02d}:{m:02d}:{sec:02d},{ms:03d}"
    
    with open(output_path, "w", encoding="utf-8") as f:
        for i, seg in enumerate(segments, 1):
            f.write(f"{i}\n{fmt_time(seg['start'])} --> {fmt_time(seg['end'])}\n{seg['translated']}\n\n")
    return output_path

print("✅ 함수 로드 완료!")

## 4️⃣ 더빙 실행 🎬

In [ ]:
#@title 더빙 설정
youtube_url = "https://www.youtube.com/watch?v=-dtGzB4zqlc" #@param {type:"string"}
voice = "female_1" #@param ["female_1", "female_2", "male_1", "male_2"]
tone = "formal" #@param ["formal", "casual", "narration"]
original_volume = 0.15 #@param {type:"slider", min:0, max:0.5, step:0.05}

import time
start_time = time.time()

# 임시 폴더
temp_dir = tempfile.mkdtemp()
print(f"📁 임시 폴더: {temp_dir}\n")

# 1. 유튜브 다운로드
print("[1/6] 📥 유튜브 다운로드")
video_path = download_youtube(youtube_url, temp_dir)
print()

# 2. 오디오 추출
print("[2/6] 🎵 오디오 추출")
audio_mp3 = f"{temp_dir}/audio.mp3"
extract_audio(video_path, audio_mp3)
print("  완료\n")

# 3. Whisper (VAD + 텍스트 추출)
print("[3/6] 🎤 음성 인식 (Whisper)")
segments = transcribe_with_timestamps(audio_mp3)
print()

# 4. 번역
print("[4/6] 🌏 번역")
segments = translate_segments(segments, tone)
print()

# 5. TTS
print("[5/6] 🔊 TTS 생성")
segments = process_tts(segments, temp_dir, voice)
segments = sync_tts_to_segments(segments, temp_dir)
print()

# 6. 최종 합성
print("[6/6] 🎬 최종 합성")
mixed_audio = f"{temp_dir}/mixed.mp3"
mix_audio(audio_mp3, segments, mixed_audio, original_volume)

output_video = "/content/dubbed_output.mp4"
output_srt = "/content/dubbed_output.srt"
replace_audio(video_path, mixed_audio, output_video)
generate_srt(segments, output_srt)
print()

elapsed = time.time() - start_time
print("="*50)
print(f"✅ 더빙 완료!")
print(f"⏱️ 소요시간: {int(elapsed//60)}분 {int(elapsed%60)}초")
print(f"📹 영상: {output_video}")
print(f"📝 자막: {output_srt}")
print("="*50)

## 5️⃣ 결과 확인 & 다운로드

In [ ]:
# 영상 미리보기
from IPython.display import Video, display
print("📺 결과 미리보기:")
display(Video('/content/dubbed_output.mp4', width=640))

In [ ]:
# 파일 다운로드
from google.colab import files

print("📥 다운로드 시작...")
files.download('/content/dubbed_output.mp4')
files.download('/content/dubbed_output.srt')
print("✅ 다운로드 완료!")